In [ ]:
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import pyfirmata
import numpy as np
​
board = pyfirmata.Arduino('/dev/cu.usbmodem11201') # Type Port Name here
it = pyfirmata.util.Iterator(board)
it.start()
analogInput = board.get_pin('a:0:i') # Type pin number here, A0 input = a:0:i

lag = 100
threshold = 2.5
influence = 0

figure, ax = plt.subplots(nrows=1, ncols=1)
plt.title('This is the Title!')
plt.xlabel('Time!')
plt.ylabel('Voltage!')
windowSize = 75 # Type Window Size here
ax.set_xlim([0, windowSize])
ax.set_ylim([0, 1.1])
​
line1, = ax.plot([], [], 'b-')
line2, = ax.plot([], [], 'r-')
line3, = ax.plot([], [], 'g-')
line4, = ax.plot([], [], 'k-')
line5, = ax.plot([], [], 'k-')
linesToPlot = [line1, line2, line3, line4, line5]
names = ['Raw Data', 'Peak Detection', 'Average', 'Lower Bound', 'Upper Bound']
ax.legend(lines, names, loc = 4)
​
def update(frame):
  volts = analogInput.read()
  rawData.append(volts)
  axis = np.arange(frame)

  if frame < lag:
    for i in range(5):
      graphVariables[i].append(0)
    for i in range(5):
      linesToPlot.set_ydata(graphVariables[i][0:frame])
  
  else:
    peaks = real_time_peak_detection(rawData, lag, threshold, influence)
    avg.append(peaks.thresholding_algo(volts)[1])
    stdFilter = peaks.thresholding_algo(volts)[2]
    lowerBound.append(avg[frame] - threshold * stdFilter)
    upperBound.append(avg[frame] - threshold * stdFilter)

    if volts >= upperBound[frame]:
      peakData.append(1)
    else:
      peakData.append(0)
    
    for i in range(5):
      linesToPlot[i].set_ydata(graphVariables[i][0:frame])
  
  if frame > windowSize:
    lin = ax.set_xlim(frame - windowSize, frame)

class real_time_peak_detection():
    def __init__(self, array, lag, threshold, influence):
        self.y = list(array)
        self.length = len(self.y)
        self.lag = lag
        self.threshold = threshold
        self.influence = influence
        self.signals = [0] * len(self.y)
        self.filteredY = np.array(self.y).tolist()
        self.avgFilter = [0] * len(self.y)
        self.stdFilter = [0] * len(self.y)
        self.avgFilter[self.lag - 1] = np.mean(self.y[0:self.lag]).tolist()
        self.stdFilter[self.lag - 1] = np.std(self.y[0:self.lag]).tolist()

    def thresholding_algo(self, new_value):
        self.y.append(new_value)
        i = len(self.y) - 1
        self.length = len(self.y)
        if i < self.lag:
            return 0
        elif i == self.lag:
            self.signals = [0] * len(self.y)
            self.filteredY = np.array(self.y).tolist()
            self.avgFilter = [0] * len(self.y)
            self.stdFilter = [0] * len(self.y)
            self.avgFilter[self.lag] = np.mean(self.y[0:self.lag]).tolist()
            self.stdFilter[self.lag] = np.std(self.y[0:self.lag]).tolist()
            return 0

        self.signals += [0]
        self.filteredY += [0]
        self.avgFilter += [0]
        self.stdFilter += [0]

        if abs(self.y[i] - self.avgFilter[i - 1]) > (self.threshold * self.stdFilter[i - 1]):

            if self.y[i] > self.avgFilter[i - 1]:
                self.signals[i] = 1
            else:
                self.signals[i] = -1

            self.filteredY[i] = self.influence * self.y[i] + \
                (1 - self.influence) * self.filteredY[i - 1]
            self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
            self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])
        else:
            self.signals[i] = 0
            self.filteredY[i] = self.y[i]
            self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
            self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])

        return self.avgFilter[i]    
​
# Create Plotter:
animation = FuncAnimation(figure, update, interval=10, blit = False)
​
plt.show()

SyntaxError: ignored